# Using PyNIO to examine NetCDF, GRIB, and HDF-EOS files

In [ ]:
import Nio as nio

### NetCDF file with a rectilinear grid

In [ ]:
filename="../Data/ts_Amon_CESM1-CAM5_historical_r1i1p1_185001-200512.nc"
f = nio.open_file(filename)
print(f)

### WRF ouput file (NetCDF) with a curvilinear grid

The WRF file doesn't have a ".nc" suffix; watch what happens if you forget to add it.

In [ ]:
# This code will produce an error about an invalid extension
filename="Data/wrfout_d03_2012-04-22_23_00_00"
f = nio.open_file(filename)
print(f)

### Add the ".nc" to the WRF filename

In [ ]:
filename="../Data/wrfout_d03_2012-04-22_23_00_00.nc"
f = nio.open_file(filename)
print(f)

### GRIB1 file - curvilinear grid

In [ ]:
filename="../Data/ruc.grb"
f = nio.open_file(filename)
print(f)

### GRIB2 file - curvilinear grid

In [ ]:
filename="../Data/MET9_IR108_cosmode_0909210000.grb2"
f = nio.open_file(filename)
print(f)

### HDF-EOS file with an ".hdf" suffix
Note the top part of the "global attributes" output.

In [ ]:
filename="../Data/MOD06_L2.A2010031.1430.005.2010031221343.hdf"
f = nio.open_file(filename)
print(f)

### Add ".he2" suffix and open file again

In [ ]:
filename = filename + ".he2"
f = nio.open_file(filename)
print(f)

### Exercise: Write short python script to output contents of any supported file, by passing the filename via a command line argument

For example:

python niodump.py -f MOD06_L2.A2010031.1430.005.2010031221343.hdf

## Open a NetCDF file and read a rectilinear grid

### Import required modules

In [ ]:
import Nio as nio
import numpy as np

In [ ]:
filename = "../Data/ts_Amon_CESM1-CAM5_historical_r1i1p1_185001-200512_partial.nc"
f   = nio.open_file(filename)
#print(f)

### Print all the variables on the file

In [ ]:
vnames = f.variables.keys()
print vnames

### Read ts, lat, lon off the file

In [ ]:
ts  = f.variables["ts"]           # Get the "ts" variable object                          
lat = f.variables["lat"][:]       # [:] reads the values                            
lon = f.variables["lon"][:]

### Print some info about variables

In [ ]:
print "ts type",type(ts)
print "lat type",type(lat)
print "lon type",type(lon)
print "ts dimension sizes",ts.shape
print "lat dimension sizes",lat.shape
print "lon dimension sizes",lon.shape

### Print attributes and dimension names of ts

In [ ]:
print "Attributes",     ts.attributes
print "Dimension names",ts.dimensions

### Accessing individual attributes and dimension names

In [ ]:
print "Variable is %s (%s)" % (ts.long_name,ts.units)
print "Name of dimension 0 is", ts.dimensions[0]

### Use [:] to access values

In [ ]:
tvals = ts[:]             # Reads all dimensions, all values
print "type",type(tvals)
print "shape",tvals.shape
print "dtype = %s" % tvals.dtype
print tvals

In [ ]:
# Two ways to get the min/max
print "min / max = %g / %g" % (np.min(tvals),np.max(tvals))
print "min / max = %g / %g" % (tvals.min(),tvals.max())
print "average = ",np.average(tvals)

### Create a contour plot using PyNGL

In [ ]:
from plot_ts_Amon_ngl import *
plot_ts_Amon_ngl(ts,lat,lon)
from IPython.display import Image
Image(filename='plot_ts_Amon.png')

### Subset "ts" using coordinate subscripting

In [ ]:
ts  = f.variables["ts"]['time|i0 lat|0:60 lon|45:135']
print ts.shape

#### Exercise: use coordinate subscripting to subscript lat/lon arrays using the same lat/lon range as "ts", and plot using "plot_ts_Amon_subset_ngl".
(Note: the plotting code will be similar to the one above, but the routine name and the PNG file name are different.)

## Open a WRF output file with a curvilinear grid

### Import required modules

In [ ]:
import Nio as nio
import numpy as np

### Open the file

In [ ]:
# Don't forget to add the ".nc"
filename="../Data/wrfout_d03_2012-04-22_23_00_00.nc"
f = nio.open_file(filename)
#print(f)

### Print all the variables on the file

In [ ]:
vnames = f.variables.keys()
print vnames

### Read "HGT" variable off file

In [ ]:
h = f.variables["HGT"]

### Print some information about "h"

In [ ]:
print "type",type(h)
print "dimension sizes",h.shape

### Print attributes and dimension names of "h"

In [ ]:
print "Attributes",     h.attributes
print "Dimension names",h.dimensions

### Accessing individual attributes and dimension names

In [ ]:
print "Variable is %s (%s)" % (h.description,h.units)
print "Name of dimension 0 is", h.dimensions[0]

### Use [:] to access values

In [ ]:
hvals = h[:]             # Reads all dimensions, all values
print "type",type(hvals)
print "shape",hvals.shape
print "dtype = %s" % hvals.dtype
print hvals

In [ ]:
# Two ways to get the min/max
print "min / max = %g / %g" % (np.min(hvals),np.max(hvals))
print "min / max = %g / %g" % (hvals.min(),hvals.max())
print "average = ",np.average(hvals)

## Reread "hgt" variable as numpy array
"h" is a 3D variable (time,lat,lon), so be sure to subscript it.

In [ ]:
h = f.variables["HGT"][0,:,:]
print "type(h)",type(h)
print "h.shape",h.shape
print h.min(),h.max()

## Create a color contour plot using PyNGL

In [ ]:
from wrf_plot_ngl import *
wrf_plot_var(h,contour_type="raster")
#wrf_plot_from_file(filename,"HGT",contour_type="raster")
from IPython.display import Image
Image(filename='wrfplot.png')

## Create a contour plot using matplotlib

In [ ]:
%matplotlib inline
from wrf_plot_hgt_mpl import WRF_Plotter
wplt = WRF_Plotter(fname = "../Data/wrfout_d03_2012-04-22_23_00_00", vname = "HGT")
wplt.plot(varname = 'HGT')
wplt.close()

### Exercise: read and plot "T2" using wrf_plot_from_file, or wrf_plot_var, or wrf_plot_T2_mpl to plot.

## Examine an HDF-EOS file with missing data

### Import required modules

In [ ]:
import Nio as nio
import numpy as np

In [ ]:
filename="../Data/MOD06_L2.A2010031.1430.005.2010031221343.hdf"
f = nio.open_file(filename)

In [ ]:
print(f)

### Read "Cloud_Top_Temperature" and print some information

In [ ]:
ctt = f.variables["Cloud_Top_Temperature"]
print ctt.shape
print ctt
print "min/max CTT = %g / %g" % (np.min(ctt[:]) ,np.max(ctt[:]))

### Note the "scale_factor" and "add_offset" attributes

In [ ]:
print "scale_factor",ctt.scale_factor
print "add_offset",ctt.add_offset

### Apply these attributes to unpack the data

In [ ]:
cttf = ctt.scale_factor * (ctt[:] - ctt.add_offset)

print "min/max CTT = %g / %g" % (np.min(cttf) ,np.max(cttf))

### What happens if we take the average?

In [ ]:
print "average = ",np.average(cttf)

### Note that cttf has a fill value

In [ ]:
print ctt._FillValue         # This is a NioVariable
print cttf.get_fill_value()  # This is a MaskedArray

### You can test for a MaskedArray

In [ ]:
print "Is cttf a MaskedArray?",isinstance(cttf, np.ma.MaskedArray)

### We have a masked array, so use "ma" module to take average

In [ ]:
import numpy.ma as ma
print "average = ",ma.average(cttf)

### This will work too

In [ ]:
print "average = ",cttf.mean()

### Create a color contour plot

In [ ]:
from plot_ctt_ngl import *

# Read lat/lon off the file for plotting                                                  
lat = f.variables["Latitude"][:]
lon = f.variables["Longitude"][:]

title = "%s (%s)" % (ctt.hdf_name,ctt.units)
msg_val = float(ctt._FillValue)
plot_ctt(cttf,msg_val,lat,lon)

In [ ]:
from IPython.display import Image
Image(filename='ctt_plot.png')

### Open WRF output file and read some variables

In [ ]:
import numpy, Nio, Ngl, os

filename = "../Data/wrfout_d03_2012-04-22_23_00_00.nc"
f  = Nio.open_file(filename,"w")      # Open WRF output file as "write"
T  = f.variables["T"]                 # potential temperature
P  = f.variables["P"]                 # perturbation pressure
PB = f.variables["PB"]                # base pressure
print T     # These are NioVariable objects
print P
print PB

### Calculate temperature from T, P, PB using "wrf_tk" function

In [ ]:
THETA = T[:] + 300          # perturbation + reference temperature
PPB   = P[:] + PB[:]
TK    = Ngl.wrf_tk(PPB, THETA)

print "TK min/max = %g / %g" % (numpy.min(TK[:]),numpy.max(TK[:]))

# Write TK to same NetCDF file

### Define variable on file using another variable's dimensions

In [ ]:
new_var_name = "TK"
f.create_variable(new_var_name,'d',T.dimensions)

### Write attributes of variable to file, changing "description" and "units"

In [ ]:
varAtts = T.__dict__.keys()    # Get all of T's attributes
varAtts.sort()
for att in varAtts:
    if att is "description":
        value = "Temperature"
    elif att is "units":
        value = "degK"      # Better than just "K"
    else:
        value = getattr(T,att)
        
    setattr(f.variables[new_var_name],att,value)   # Write attribute and value to file

### Write data values to file and close it

In [ ]:
f.variables[new_var_name].assign_value(TK)
f.close()

### Reopen file and print it

In [ ]:
f2 = Nio.open_file(filename,"r")
print f2

### Read TK

In [ ]:
tk = f2.variables["TK"]
print "TK after reading back in: %g / %g" % (numpy.min(tk[:]),numpy.max(tk[:]))
print TK.shape

### Plot TK directly from file

In [ ]:
from wrf_plot_ngl import *
wrf_plot_from_file(filename,"TK")
from IPython.display import Image
Image(filename='wrfplot.png')